In [8]:
import networkx as nx
import numpy as np
import drawing_general as draw
import energy_new as en
import ising
from tqdm import tqdm_notebook as tqdm
import matplotlib.pyplot as plt
from stats import *
from random import choices 
import preprocessing_gen as pre

In [9]:
config_file = 'sample_config_gen.txt'
dual_graph, data_vectors, pos, node_size = pre.get_data(config_file)

/Users/Heather/miniconda2/envs/vrdi/lib/python3.6/site-packages/pysal/weights/weights.py:189: UserWarning: There are 2 disconnected observations
  warnings.warn("There are %d disconnected observations" % ni)
/Users/Heather/miniconda2/envs/vrdi/lib/python3.6/site-packages/pysal/weights/weights.py:190: UserWarning: Island ids: 197, 109
  warnings.warn("Island ids: %s" % ', '.join(str(island) for island in self.islands))


In [14]:
for j in range(13):
    running_total_score = 0
    print(j)
    min_score = 1
    max_score = 0 
    for i in range(30):
        repvote = data_vectors[2*j+1]
        demvote = data_vectors[2*j+2]
        repvote_copy = np.copy(repvote)
        demvote_copy = np.copy(demvote)
        #vector of the total number of votes in each precinct, indexed in order of nodelist 
        total = repvote_copy + demvote_copy 
        #total population 
        pop_count = np.sum(total)
        #choose node according to its weight
        #drop a certain, fixed amount on it 
        min_prop = np.sum(repvote_copy)/pop_count 
        drop_number = 10
        population_weights = []
        for i in range(total.size):
            population_weights.append(total[i]/pop_count)
        #dictionary of minority population 
        d = {}
        for node in dual_graph.nodes():
            d[node]=0
        #create a dictionary of the indices of nodes 
        indices = {}
        count = 0 
        for node in dual_graph.nodes():
            indices[node]=count 
            count += 1
        states = [node for node in dual_graph.nodes()]
        count = 0 
        while count < min_prop*pop_count:
            node_choice = choices(states, population_weights)[0]
            if d[node_choice] < total[indices[node_choice]] - drop_number:
                d[node_choice] += drop_number
                count += drop_number
                #print(count)
            else:
                old = d[node_choice]
                d[node_choice] = total[indices[node_choice]]
                count += d[node_choice] - old
        idx = 0
        for node in dual_graph.nodes():
            repvote_copy[idx]=d[node]
            idx += 1 
        A = nx.to_scipy_sparse_matrix(dual_graph)
        capy_score = en.sparse_inner_capy(repvote_copy, total-repvote_copy, A)
        #print(capy_score)
        min_score = min(min_score, capy_score)
        max_score = max(max_score, capy_score)
        running_total_score += capy_score 
    print(running_total_score/30)
    print(min_score)
    print(max_score)

0
[[0.50006938]]
[[0.5000215]]
[[0.50010434]]
1
[[0.50005537]]
[[0.50001745]]
[[0.5000808]]
2
[[0.5000693]]
[[0.50003846]]
[[0.50010582]]
3
[[0.50005929]]
[[0.50004049]]
[[0.50009873]]
4
[[0.500075]]
[[0.50003339]]
[[0.50010802]]
5
[[0.50005287]]
[[0.50002919]]
[[0.50008534]]
6
[[0.50005665]]
[[0.50003638]]
[[0.50008996]]
7
[[0.50010274]]
[[0.50003882]]
[[0.50017237]]
8
[[0.50005284]]
[[0.50003524]]
[[0.50007223]]
9
[[0.5000574]]
[[0.50003229]]
[[0.50008421]]
10
[[0.50015826]]
[[0.50008047]]
[[0.50028837]]
11
[[0.50008721]]
[[0.5000512]]
[[0.50013074]]
12
[[0.50004996]]
[[0.50002927]]
[[0.50007933]]
